### Install package

In [1]:
!pip install pytorch-adapt

     |████████████████████████████████| 137 kB 12.6 MB/s 
     |████████████████████████████████| 397 kB 44.7 MB/s 
     |████████████████████████████████| 107 kB 51.1 MB/s 


### Import packages

In [2]:
import torch
from tqdm import tqdm

from pytorch_adapt.containers import Models, Optimizers
from pytorch_adapt.datasets import DataloaderCreator, get_mnist_mnistm
from pytorch_adapt.hooks import DANNHook
from pytorch_adapt.models import Discriminator, mnistC, mnistG
from pytorch_adapt.utils.common_functions import batch_to_device
from pytorch_adapt.validators import IMValidator

### Create datasets and dataloaders

In [3]:
datasets = get_mnist_mnistm(["mnist"], ["mnistm"], folder=".", download=True)
dc = DataloaderCreator(batch_size=32, num_workers=2)
dataloaders = dc(**datasets)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./MNIST/raw/train-images-idx3-ubyte.gz to ./MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./MNIST/raw/train-labels-idx1-ubyte.gz to ./MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./MNIST/raw/t10k-images-idx3-ubyte.gz to ./MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./MNIST/raw/t10k-labels-idx1-ubyte.gz to ./MNIST/raw



  0%|          | 0/134178716 [00:00<?, ?it/s]

100%|██████████| 68007/68007 [00:17<00:00, 3817.40it/s]


### Create models, optimizers, hook, and validator

In [4]:
device = torch.device("cuda")

G = mnistG(pretrained=True).to(device)
C = mnistC(pretrained=True).to(device)
D = Discriminator(in_size=1200, h=256).to(device)
models = Models({"G": G, "C": C, "D": D})
optimizers = Optimizers((torch.optim.Adam, {"lr": 0.0001}))
optimizers.create_with(models)
optimizers = list(optimizers.values())

hook = DANNHook(optimizers)
validator = IMValidator()

Downloading: "https://cornell.box.com/shared/static/tdx0ts24e273j7mf3r2ox7a12xh4fdfy" to /root/.cache/torch/hub/checkpoints/mnistG-68ee7945.pth


  0%|          | 0.00/161k [00:00<?, ?B/s]

Downloading: "https://cornell.box.com/shared/static/j4zrogronmievq1csulrkai7zjm27gcq" to /root/.cache/torch/hub/checkpoints/mnistC-ac7b5a13.pth


  0%|          | 0.00/1.31M [00:00<?, ?B/s]

### Train and evaluate

In [5]:
for epoch in range(2):

    # train loop
    models.train()
    for data in tqdm(dataloaders["train"]):
        data = batch_to_device(data, device)
        _, loss = hook({**models, **data})

    # eval loop
    models.eval()
    logits = []
    with torch.no_grad():
        for data in tqdm(dataloaders["target_train"]):
            data = batch_to_device(data, device)
            logits.append(C(G(data["target_imgs"])))
        logits = torch.cat(logits, dim=0)

    # validation score
    score = validator(target_train={"logits": logits})
    print(f"\nEpoch {epoch} score = {score}\n")

100%|██████████| 1844/1844 [00:27<00:00, 65.96it/s]



Epoch 0 score = 1.0747219324111938



100%|██████████| 1844/1844 [00:27<00:00, 65.88it/s]


Epoch 1 score = 1.31474369764328

